In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from IPython.display import clear_output, display
from datetime import datetime
import traceback
import time

In [ ]:
URL = 'https://www.tradingview.com/chart/CODE/'
USERNAME = 'your.email@mail.com'
PASS = 'YourPassword'

try:
    from dev_settings import *
except ImportError:
    pass

In [ ]:
option = webdriver.ChromeOptions()
#browser = webdriver.Chrome('/usr/local/bin/chromedriver')
browser = webdriver.Chrome(chrome_options=option)

# go to website of interest
browser.get(URL + '#signin')
username = browser.find_element_by_name("username")
username.send_keys(USERNAME)
password = browser.find_element_by_name("password")
password.send_keys(PASS)
password.send_keys(Keys.RETURN);


In [ ]:
def click(id=3):
    """Click on canvas to get cross on middle of graph"""
    canvases = browser.find_elements_by_xpath("//canvas")
    try:
        canvases[id].click()
        print("click")
    except:
        pass
    
def moveRight(n = 1):
    """Move to next n days"""
    for i in range(n):
        ActionChains(browser)\
        .key_down(Keys.ARROW_RIGHT).perform()

def moveLeft(n = 1):
    """Move to previous n days"""
    for i in range(n):
        ActionChains(browser)\
        .key_down(Keys.ARROW_LEFT).perform()

def getTime(ts):
    """conver unix time to python datetime"""
    ts = int(ts)
    dt = datetime.utcfromtimestamp(ts)
    print(dt)
    return dt

def getPrice():
    """get current price [Open, High, Low, Close, Volumn, UnixTimeStamp]"""
    attempts = 0;
    # retry 100 attempts with 5 sec between each attemts
    while(attempts < 100):
        attempts += 1
        try:
            #get value from indicator canvas where color == rgb(128,128,0,1)
            prices_DOM = browser.find_elements_by_xpath("//span[@class='pane-legend-item-value' and contains(@style,color)]")
            prices = [float(price.text) for price in prices_DOM \
            if price.value_of_css_property("color")=='rgba(128, 128, 0, 1)']
            return prices
        except:
            print('retry fetch price -> ',attempts, 'attempts')
            time.sleep(.3)
            pass
    #error
    return 0

def appendFile(file, p):
    """append price from getPrice() to file"""
    with open(file, "a") as myfile:
            myfile.write("{},{},{},{},{},{}\n".format(p[0],p[1],p[2],p[3],p[4],p[5]))

In [ ]:
currentTime = 9999999999
price_all = []
i = 0
halt = False
click()

while not halt:
    moveLeft()
    p = getPrice()
    if currentTime > p[5]:
        price_all.append(p)
        appendFile("data.csv", p)
        
        i += 1
        currentTime = p[5]
        if i % 10 == 0:
            clear_output()
        print(i, "open", p[1] , end=" ")
        getTime(p[5])
    else:
        print('Duplicate Data')
        click()
        time.sleep(1)
        oldp = ''
        while True:
            p = getPrice()
            if oldp == p:
                click()
                time.sleep(1)
            if currentTime < p[5]:
                moveLeft()
            else:
                break
            oldp = p